<a href="https://colab.research.google.com/github/rosanth/Recommended_Sys/blob/Sajeevan/test_re_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf
!pip install pyngrok
!pip install flask pyngrok
from flask import Flask, request, jsonify, render_template_string
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import os
import re
import fitz
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
# HTML code embedded in Flask app
html_code = '''


'''

In [ ]:


# Initialize Flask app
app = Flask(__name__)

# Load Pretrained Sentence Embeddings Model (BERT-based)
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Function to extract resume text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return preprocess_text(text.strip())

# Function to compute similarity
def compute_similarity(pdf_path, job_desc):
    job_embeddings = np.array([bert_model.encode(preprocess_text(job)) for job in job_desc])
    resume_text = extract_text_from_pdf(pdf_path)
    resume_embedding = bert_model.encode(resume_text)
    real_resume_similarity = cosine_similarity([resume_embedding], job_embeddings)[0]
    sorted_jobs = np.argsort(-real_resume_similarity)

    # Prepare response
    results = []
    for idx in sorted_jobs:
        results.append({"job_title": job_desc[idx], "similarity_score": float(real_resume_similarity[idx])})
    return results



# Route to render the HTML
@app.route("/")
def home():
    return render_template_string(html_code)

# Route to handle file upload and process resume
@app.route("/upload", methods=["POST"])
def upload():
    if "resume" not in request.files or "job_descriptions" not in request.form:
        return jsonify({"error": "Missing file or job descriptions"}), 400
    file = request.files["resume"]
    job_descriptions = request.form["job_descriptions"].splitlines()

    # Save the file temporarily
    filename = secure_filename(file.filename)
    filepath = os.path.join("/tmp", filename)
    file.save(filepath)

    # Compute similarity
    results = compute_similarity(filepath, job_descriptions)

    # Render results in the HTML
    return render_template_string(html_code, results=results)

# ============================ NGROK SETUP ============================ #
# Add your ngrok authtoken (Replace 'YOUR_AUTHTOKEN' with actual token)
NGROK_AUTH_TOKEN = "2uB8nIyyiphrSbk0IZgp3KTFIqR_4pdXPSnZ45FdABkqkjbJq"
os.system(f"ngrok config add-authtoken {NGROK_AUTH_TOKEN}")

# Start ngrok tunnel to expose the app
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

# Run Flask app
if __name__ == "__main__":
    app.run(port=5000)
